<a href="https://colab.research.google.com/github/dev-SR/Deep-Learning/blob/main/04-pytorch-lighting-intro/pl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 57.0 MB/s eta 0:00:00


In [2]:
%%capture
!pip install datasets

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from torchmetrics import Accuracy
from datasets import load_dataset
import numpy as np
import pandas as pd

# ignore warning
import warnings

warnings.filterwarnings("ignore")

In [34]:
class MultiLayerPerceptron(pl.LightningModule):
    def __init__(self, image_shape=(3, 28, 28), hidden_units=(32, 16)):
        super().__init__()

        # new PL attributes:
        self.train_acc = Accuracy(task="multiclass", num_classes=10)
        self.valid_acc = Accuracy(task="multiclass", num_classes=10)
        self.test_acc = Accuracy(task="multiclass", num_classes=10)

        # Model similar to previous section:
        input_size = image_shape[0] * image_shape[1] * image_shape[2]
        all_layers = [nn.Flatten()]
        for hidden_unit in hidden_units:
            layer = nn.Linear(input_size, hidden_unit)
            all_layers.append(layer)
            all_layers.append(nn.ReLU())
            input_size = hidden_unit

        all_layers.append(nn.Linear(hidden_units[-1], 10))
        self.model = nn.Sequential(*all_layers)

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
def training_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    loss = nn.functional.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim=1)
    self.train_acc.update(preds, y)
    self.log("train_loss", loss, prog_bar=True)
    return loss


def training_epoch_end(self, outs):
    self.log("train_acc", self.train_acc.compute())
    self.train_acc.reset()


def validation_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    loss = nn.functional.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim=1)
    self.valid_acc.update(preds, y)
    self.log("valid_loss", loss, prog_bar=True)
    return loss


def validation_epoch_end(self, outs):
    self.log("valid_acc", self.valid_acc.compute(), prog_bar=True)
    self.valid_acc.reset()


def test_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    loss = nn.functional.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim=1)
    self.test_acc.update(preds, y)
    self.log("test_loss", loss, prog_bar=True)
    self.log("test_acc", self.test_acc.compute(), prog_bar=True)
    return loss


def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
    return optimizer

In [4]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [6]:
np.__version__

'1.23.5'

In [7]:
class MyDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.images = dataset["img"]
        self.labels = dataset["label"]
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            augmented = self.transform(image=np.array(image))
            image = augmented["image"]
        return {
            "image": image,
            "label": label,
        }

In [33]:
# define reusable config
CONFIG = dict(
    batch_size=32,
    img_size=32,
    hidden_units=(32, 16),
    max_epochs=10,
    gpus=1 if torch.cuda.is_available() else 0,
)


class MnistDataModule(pl.LightningDataModule):
    def __init__(self, batch_size):
        super().__init__()
        self.batch_size = batch_size
        # Train augmentation policy
        self.train_transform = A.Compose(
            [
                A.RandomResizedCrop(
                    height=CONFIG["img_size"], width=CONFIG["img_size"]
                ),
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(p=0.5),
                A.RandomBrightnessContrast(p=0.5),
                A.Normalize(),
                ToTensorV2(),
            ]
        )

        # Validation/Test augmentation policy
        self.test_transform = A.Compose(
            [
                A.Resize(height=CONFIG["img_size"], width=CONFIG["img_size"]),
                A.Normalize(),
                ToTensorV2(),
            ]
        )

    def prepare_data(self):
        self.train_ds, self.test_ds = load_dataset(
            "cifar10", split=["train[:1000]", "test[:200]"]
        )
        print(self.train_ds)
        print(np.array(self.train_ds['img'][0]).shape)

    def setup(self, stage=None):
        # stage is either 'fit', 'validate', 'test', or 'predict'
        self.train = MyDataset(self.train_ds, self.train_transform)
        self.valid = MyDataset(self.test_ds, self.test_transform)
        self.test = MyDataset(self.test_ds, self.test_transform)

    def train_dataloader(self):
        return DataLoader(
            self.train,
            batch_size=self.batch_size,
            num_workers=4,
            shuffle=True,
            )

    def val_dataloader(self):
        return DataLoader(
            self.valid,
            batch_size=self.batch_size,
            num_workers=4,
            )

    def test_dataloader(self):
        return DataLoader(
            self.test,
            batch_size=self.batch_size,
            num_workers=4,
            )

In [32]:
datamodule = MnistDataModule(batch_size=CONFIG["batch_size"])
datamodule.prepare_data()
datamodule.setup()
# Samples required by the custom ImagePredictionLogger callback to log image predictions.
val_samples = next(iter(datamodule.val_dataloader()))
val_imgs, val_labels = val_samples["image"], val_samples["label"]
val_imgs.shape, val_labels.shape

Dataset({
    features: ['img', 'label'],
    num_rows: 1000
})
(32, 32, 3)


(torch.Size([32, 3, 64, 64]), torch.Size([32]))